In [1]:
import warnings                            # This block prevents display of harmless warnings, but should be
warnings.filterwarnings('ignore')          # commented out till the final version, to avoid missing "real" warnings 
import kilroy_was_here                     # Mandatory. Allow access to shared python code in the upper 'lib' directory
from lib.audiodataset import AudioDataset  # Class for audio dataset handling

# Dataset name is the master key for dataset adressing
DATASET_NAME = 'SMALL1005'

# Initialize Dataset Object. By NOT providing a source path, you implicitly express
# express the intent of RETRIEVING an existing dataset rather than CREATING a new one
ds = AudioDataset(DATASET_NAME)

ds.info()

[2020-08-06/19:57:40.532|14.4%|74.6%|0.25GB] ------------------------------------------------------
[2020-08-06/19:57:40.532|00.0%|74.6%|0.25GB] DATASET PATH          : D:\Jupyter\ShowBees\datasets\SMALL1005
[2020-08-06/19:57:40.532|00.0%|74.6%|0.25GB] DATASET DB PATH       : D:\Jupyter\ShowBees\datasets\SMALL1005\database.db
[2020-08-06/19:57:40.532|00.0%|74.6%|0.25GB] DATASET SAMPLES PATH  : D:\Jupyter\ShowBees\datasets\SMALL1005\samples
[2020-08-06/19:57:40.532|00.0%|74.6%|0.25GB] NB SOURCE AUDIO FILES : 4
[2020-08-06/19:57:40.532|00.0%|74.6%|0.25GB] SAMPLE RATE           : 22050
[2020-08-06/19:57:40.532|00.0%|74.6%|0.25GB] DURATION              : 1.0
[2020-08-06/19:57:40.532|00.0%|74.6%|0.25GB] OVERLAP               : 0.5
[2020-08-06/19:57:40.532|00.0%|74.6%|0.25GB] NB AUDIO CHUNKS       : 4744
[2020-08-06/19:57:40.532|00.0%|74.6%|0.25GB] ------------------------------------------------------


In [2]:
from lib import attributors
print(ds.getAttrNames())  
ds.setAttr('prout', attributors.FromString('hello'))
print(ds.getAttrNames())  
ds.dropAttr('prout')
print(ds.getAttrNames())   


['hive']
['hive', 'prout']
['hive']


In [3]:
sql = """SELECT s.rowid, s.name FROM samples s INNER JOIN attributes a
          ON s.rowid = a.sample_id
          WHERE a.name = 'hive' and a.value = ?"""
ds.query(sql, ('Hive1',), as_dict=False)

(1, '00-000000')
(2, '00-000001')
(3, '00-000002')
(4, '00-000003')
(5, '00-000004')
(6, '00-000005')
(7, '00-000006')
(8, '00-000007')
(9, '00-000008')
(10, '00-000009')
(11, '00-000010')
(12, '00-000011')
(13, '00-000012')
(14, '00-000013')
(15, '00-000014')
(16, '00-000015')
(17, '00-000016')
(18, '00-000017')
(19, '00-000018')
(20, '00-000019')
(21, '00-000020')
(22, '00-000021')
(23, '00-000022')
(24, '00-000023')
(25, '00-000024')
(26, '00-000025')
(27, '00-000026')
(28, '00-000027')
(29, '00-000028')
(30, '00-000029')
(31, '00-000030')
(32, '00-000031')
(33, '00-000032')
(34, '00-000033')
(35, '00-000034')
(36, '00-000035')
(37, '00-000036')
(38, '00-000037')
(39, '00-000038')
(40, '00-000039')
(41, '00-000040')
(42, '00-000041')
(43, '00-000042')
(44, '00-000043')
(45, '00-000044')
(46, '00-000045')
(47, '00-000046')
(48, '00-000047')
(49, '00-000048')
(50, '00-000049')
(51, '00-000050')
(52, '00-000051')
(53, '00-000052')
(54, '00-000053')
(55, '00-000054')
(56, '00-000055')
(

In [4]:
sql = """SELECT count(s.rowid) FROM samples s INNER JOIN labels l
          ON s.rowid = l.sample_id
          WHERE l.name = 'nobee' and l.value = 1 and l.strength > ? """
ds.query(sql, (0.5,), as_dict=True)

{'count(s.rowid)': 1554}


In [ ]:
#List columns
SELECT name FROM PRAGMA_TABLE_INFO('your_table');


# Check from column existence
SELECT 1 FROM PRAGMA_TABLE_INFO('your_table') WHERE name='sql';
1


# add column
ALTER TABLE table_name
ADD COLUMN column_definition;

#rename column
ALTER TABLE table_name
RENAME COLUMN current_name TO new_name;

# delete column
PRAGMA foreign_keys=off;

BEGIN TRANSACTION;

CREATE TABLE IF NOT EXISTS temp (
same as source but without de dropped column
);

INSERT INTO temp(all column but the dropped one)
SELECT all column but the dropped one 
FROM table_name;

DROP TABLE table_name;

ALTER TABLE temp RENAME TO table_name;

COMMIT;

PRAGMA foreign_keys=on;


In [18]:
sql = """
select s.name, a.value as hive, l1.value as nobee, l2.value as queen from samples s, attributes a, labels as l1, labels as l2
where 1
and l1.name='nobee'
and l2.name='queen'
and a.name='hive'
and l1.sample_id = s.rowid
and l2.sample_id = s.rowid
and a.sample_id = s.rowid
"""

ds.getDataFrame(sql)

,name,hive,nobee,queen
0,00-000000,Hive1,0,1
1,00-000001,Hive1,0,1
2,00-000002,Hive1,0,1
3,00-000003,Hive1,0,1
4,00-000004,Hive1,0,1
...,...,...,...,...
4739,03-001178,Hive3,0,1
4740,03-001179,Hive3,0,1
4741,03-001180,Hive3,0,1
4742,03-001181,Hive3,0,1


In [27]:
def dumpDataFrame(samples_flds, attributes_flds, labels_flds):
    sql_select = "select s.rowid"
    sql_from = "\nfrom samples s"
    sql_where = "\nwhere 1\n"
    
    for sf in samples_flds:
        sql_select += ", s." + sf
    
    for 
        
    
    sql = sql_select + sql_from + sql_where
    
    print(sql)
    
    return ds.getDataFrame(sql)

dumpDataFrame(('name', 'file_id'), ('hive',), ('queen', 'nobee'))

select s.rowid, s.name, s.file_id
from samples s
where 1



,rowid,name,file_id
0,1,00-000000,1
1,2,00-000001,1
2,3,00-000002,1
3,4,00-000003,1
4,5,00-000004,1
...,...,...,...
4739,4740,03-001178,4
4740,4741,03-001179,4
4741,4742,03-001180,4
4742,4743,03-001181,4


In [1]:
import warnings
warnings.filterwarnings('ignore')
import sqlite3
from contextlib import closing
import kilroy_was_here
from lib import dblib
from lib.audiodataset import AudioDataset

DATASET_NAME = 'SMALL1005'
ds = AudioDataset(DATASET_NAME)

    
ds.addLabel("toto1")
print(ds.existsLabel('toto1'))
ds.addLabel("toto2")
ds.addLabel("toto3")
ds.addLabel("TOTO3")
print("Labels = ", ds.listLabels())
ds.dropLabel("toto1")
print("Labels = ", ds.listLabels())
print(existsLabel('toto1'))
ds.dropLabel("toto2")
print("Labels = ", ds.listLabels())  
ds.dropLabel("toto3")
print("Labels = ", ds.listLabels())  
    

TypeError: match() missing 1 required positional argument: 'string'